In [1]:
# Dependências

import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
# Parâmetros

TEST_SIZE = 0.2
TRAIN_SIZE = 0.8
SEED = 42

In [3]:
# Import de dados

uri = r'C:\Users\natha\Pictures\Infnet\kobe-classifier\data\01_raw\kobe_datase.csv'
data = pd.read_csv(uri)

In [4]:
print(f'Quantidade de linhas com dados faltantes: {data.shot_made_flag.isna().sum()}')
data.dropna(inplace=True)
print(f'Quantidade de linhas com dados faltantes: {data.shot_made_flag.isna().sum()}')

Quantidade de linhas com dados faltantes: 5000
Quantidade de linhas com dados faltantes: 0


In [5]:
data.columns

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id'],
      dtype='object')

coluna lng no trabalho, no dataset -> lon

In [6]:
data.groupby('shot_type').count()['game_id']

shot_type
2PT Field Goal    20285
3PT Field Goal     5412
Name: game_id, dtype: int64

Temos 20.285 inputs do shot_type = '2PT Field Goal'

In [7]:
data = data[data['shot_type'] == '2PT Field Goal']

In [8]:
columns = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']

In [9]:
data = data[columns]

In [17]:
data.shape

(20285, 7)

In [11]:
data.head()

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
1,34.0443,-118.4268,10,1,0,15,0.0
2,33.9093,-118.3708,7,1,0,16,1.0
3,33.8693,-118.1318,6,1,0,22,0.0
4,34.0443,-118.2698,6,2,0,0,1.0
5,34.0553,-118.4148,9,3,0,14,0.0


In [12]:
X = data.drop('shot_made_flag', axis = 1)
y = data['shot_made_flag'].copy()

In [13]:
X.columns

Index(['lat', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'shot_distance'],
      dtype='object')

In [14]:
y = data.drop(columns=X.columns)
y

,shot_made_flag
1,0.0
2,1.0
3,0.0
4,1.0
5,0.0
...,...
30690,0.0
30691,0.0
30692,0.0
30694,1.0


In [15]:
def prepared_data(data):
    columns = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']

    data.dropna(inplace=True)
    data = data[data['shot_type'] == '2PT Field Goal']
    data = data[columns]

    return prepared_data

# Treino e teste

In [16]:
def split_data(prepared_data):
    X = data.drop('shot_made_flag', axis = 1)
    y = data['shot_made_flag'].copy()

    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=TEST_SIZE, 
                                                        random_state=SEED,
                                                        shuffle=True,
                                                        stratify=y)
    
    return X_train, X_test, y_train, y_test